In [1]:
from current_season_code.functions import *
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import jupyterthemes as jt

In [2]:
bets = pd.read_csv('current_season_data/bets.csv', index_col = 0)
bets[['Team1', 'Team2', 'Team1_Prob', 'Team2_Prob', 'Team1_KC', 'Team2_KC', 'Team1_Bet', 'Team2_Bet']]

In [4]:
bets_external = pd.read_csv('current_season_data/bets_external.csv', index_col = 0)
bets_external[['Away_Team', 'Home_Team', 'Home_Bet', 'Away_Bet', 'Home_Bet_Combined', 'Away_Bet_Combined']]

,Away_Team,Home_Team,Home_Bet,Away_Bet,Home_Bet_Combined,Away_Bet_Combined
0,Magic,Clippers,4251.141113,0,2776.845918,0
1,Jazz,Wizards,606.642634,0,1550.046051,0
2,Bulls,Heat,2822.250100,0,2648.628364,0
3,Rockets,Grizzlies,4409.765781,0,4357.305068,0
4,Kings,Cavaliers,0.000000,0,0.000000,0
5,Nuggets,Spurs,1299.453283,0,2208.626254,0
6,Warriors,76ers,2902.579646,0,606.444811,0


In [10]:
# Getting NBA injuries

tables = pd.read_html('https://www.cbssports.com/nba/injuries/')
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.cbssports.com/nba/injuries/')
team_names = driver.find_elements_by_xpath('//*[@id="TableBase"]/h4')
table_dict = {}
for i in range(len(tables)):
    table_dict[team_names[i].text] = tables[i]
for key,value in table_dict.items():
    value['Player'] = value.Player.str.split()
    for index, row in value.iterrows():
        if len(row.Player) == 4:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2]
        if len(row.Player) == 6:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2] + ' ' + row.Player[-3]
for key,value in table_dict.items():
    value['Player'] = value.Player.str.split()
    for index, row in value.iterrows():
        if len(row.Player) == 2:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2]
        if len(row.Player) == 3:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2] + ' ' + row.Player[-3]



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [11]:
# Getting NBA stats

